In [1]:
pip install transformers torch pandas sklearn datasets


  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=728615726deb26095a307f3f36ed2a9df52b26e155093c24de7099a1c8997874
  Stored in directory: c:\users\shrav\appdata\local\pip\cache\wheels\22\0b\40\fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897
Successfully built sklearn
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.65.0
    Uninstalling tqdm-4.65.0:
      Successfully uninstalled tqdm-4.65.0
  Attempting uninstall: fsspec
    Found existing installation: fsspec 0.9.0
    Uninstalling fsspec-0.9.0:
      Successfully uninstalled fsspec-0.9.0
Note: you may need to restart the kernel to use updated packages.


    ERROR: Command errored out with exit status 1:
     command: 'c:\Users\shrav\anaconda3\python.exe' -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\shrav\\AppData\\Local\\Temp\\pip-install-p0zpoiay\\sklearn_1774f105466f458bbb7132150f3c1532\\setup.py'"'"'; __file__='"'"'C:\\Users\\shrav\\AppData\\Local\\Temp\\pip-install-p0zpoiay\\sklearn_1774f105466f458bbb7132150f3c1532\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\shrav\AppData\Local\Temp\pip-pip-egg-info-m5uzwaf5'
         cwd: C:\Users\shrav\AppData\Local\Temp\pip-install-p0zpoiay\sklearn_1774f105466f458bbb7132150f3c1532\
    Complete output (15 lines):
    The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
    rather than 'sklearn' for pip commands.
    
    Here is how to fix this error in the main use cases:
    - use 'pip install scikit-le

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments, TrainerCallback
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder
import torch
import os
import joblib
from sklearn.metrics import accuracy_score

c:\Users\shrav\anaconda3\lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
c:\Users\shrav\anaconda3\lib\site-packages\torchvision\transforms\v2\__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: h

In [3]:
df = pd.read_csv('drugsComTrain_raw.tsv', sep='\t')
len(df)

161297

In [5]:
# Load and prepare data
df = pd.read_csv('drugsComTrain_raw.tsv', sep='\t', nrows = 150000)
df = df[df['condition'].isin(['Birth Control', 'Depression', 'High Blood Pressure', 'Diabetes, Type 2'])]
df = df.dropna(subset=['review'])  # Ensuring no null reviews

# Preprocess text data
def preprocess_text(text):
    return text.str.replace('<[^<]+?>', '')  # Remove HTML tags

df['cleaned_text'] = preprocess_text(df['review'])

# Prepare labels
label_encoder = LabelEncoder()
df['encoded_labels'] = label_encoder.fit_transform(df['condition'])
labels = df['encoded_labels']

# Split data first
X_train, X_test, y_train, y_test = train_test_split(df['cleaned_text'], labels, test_size=0.2, random_state=42)

# Tokenization
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(list(X_train), truncation=True, padding=True, max_length=256)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True, max_length=256)

# Create torch datasets
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels.iloc[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(train_encodings, y_train)
test_dataset = CustomDataset(test_encodings, y_test)

# Model setup
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(label_encoder.classes_))

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
)
# Prepare to capture training and validation loss
training_loss_set = []
validation_loss_set = []

# Callback to Hugging Face Trainer to capture training loss after each logging step
class LossLoggingCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if 'loss' in logs:
            training_loss_set.append(logs['loss'])
        if 'eval_loss' in logs:
            validation_loss_set.append(logs['eval_loss'])
# Define the compute_metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc}

# Initialize Trainer with compute_metrics
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print(f"Test Loss: {results['eval_loss']}, Test Accuracy: {results['eval_accuracy']}")


<ipython-input-5-d87ea7d5dfb7>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  return text.str.replace('<[^<]+?>', '')  # Remove HTML tags
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\shrav\anaconda3\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>={ACCELERATE_MIN_VERSION}'`

In [ ]:
len(df)

39788

In [ ]:
model.save_pretrained('./distilbert-drug-review-model')
tokenizer.save_pretrained('./distilbert-drug-review-tokenizer')


('./distilbert-drug-review-tokenizer/tokenizer_config.json',
 './distilbert-drug-review-tokenizer/special_tokens_map.json',
 './distilbert-drug-review-tokenizer/vocab.txt',
 './distilbert-drug-review-tokenizer/added_tokens.json')

In [ ]:

print("Model directory contents:", os.listdir('./distilbert-drug-review-model'))
print("Tokenizer directory contents:", os.listdir('./distilbert-drug-review-tokenizer'))


Model directory contents: ['config.json', 'pytorch_model.bin']
Tokenizer directory contents: ['tokenizer_config.json', 'special_tokens_map.json', 'vocab.txt']


In [ ]:

# Load trained model and tokenizer
model_path = './distilbert-drug-review-model'
tokenizer_path = './distilbert-drug-review-tokenizer'

model = DistilBertForSequenceClassification.from_pretrained(model_path)
tokenizer = DistilBertTokenizer.from_pretrained(tokenizer_path)


In [ ]:
def predict_condition(texts):
    encoded_texts = tokenizer(texts, padding=True, truncation=True, max_length=256, return_tensors="pt")
    outputs = model(**encoded_texts)
    predictions = torch.argmax(outputs.logits, dim=-1)
    return [label_encoder.classes_[pred] for pred in predictions.tolist()]  # Adjusted for direct use of classes

# Example usage
texts = [
    "I've been on birth control for two years with no side effects.",
    "This medication made my depression worse.",
    "Excellent control of blood pressure with this medication."
]
predicted_conditions = predict_condition(texts)
for text, pred in zip(texts, predicted_conditions):
    print(f"Text: {text}\nPredicted Condition: {pred}\n")


Text: I've been on birth control for two years with no side effects.
Predicted Condition: Birth Control

Text: This medication made my depression worse.
Predicted Condition: Depression

Text: Excellent control of blood pressure with this medication.
Predicted Condition: High Blood Pressure



In [ ]:
joblib.dump(label_encoder, 'label_encoder.pkl')  # Saving
label_encoder = joblib.load('label_encoder.pkl')  # Loading


[]